In [48]:
import re
import json
import requests
import urllib.request
from bs4 import BeautifulSoup

In [31]:
# Convert CH y/x to WGS
def CHtoWGS(yx):
    y = yx[0]
    x = yx[1]
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
            + (0.791484 * y_aux * x_aux) + \
            + (0.1306 * y_aux * pow(x_aux, 2))) + \
            - (0.0436 * pow(y_aux, 3))
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = round((lng * 100) / 36, 4)
    lat = round((lat * 100) / 36, 4)
    return [lng, lat]

Flow

In [32]:
url = "https://data.geo.admin.ch/ch.bafu.hydroweb-messstationen_zustand/ch.bafu.hydroweb-messstationen_zustand_en.json"
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
del data["crs"]
del data["mapname"]
del data["timestamp"]
for station in data["features"]:
    station["geometry"]["coordinates"] = CHtoWGS(station["geometry"]["coordinates"])
    station["properties"]["link"] = re.search("href='(.*)'>Station page", station["properties"]["description"]).group(1)
    del station["properties"]["param"]
    del station["properties"]["w-typ"]
    del station["properties"]["quant-class"]
    del station["properties"]["description"]
with open('../data/flow_stations.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False)

Temperature

In [45]:
url = "https://data.geo.admin.ch/ch.bafu.hydroweb-messstationen_temperatur/ch.bafu.hydroweb-messstationen_temperatur_en.json"
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
del data["crs"]
del data["mapname"]
del data["timestamp"]
for station in data["features"]:
    station["geometry"]["coordinates"] = CHtoWGS(station["geometry"]["coordinates"])
    station["properties"]["link"] = re.search("href='(.*)'>Station page", station["properties"]["description"]).group(1)
    del station["properties"]["param"]
    del station["properties"]["w-typ"]
    del station["properties"]["temp-class"]
    del station["properties"]["description"]
    
with open('../data/zurich_stations.json') as json_file:
    zdata = json.load(json_file)
    
for z in zdata:
    z["geometry"]["coordinates"] = CHtoWGS([z["geometry"]["coordinates"][0]-2000000,z["geometry"]["coordinates"][1]-1000000])
    data["features"].append(z)

with open('../data/temperature_stations.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False)

In [46]:
data

{'features': [{'geometry': {'coordinates': [6.8886, 46.3496], 'type': 'Point'},
   'id': '2009',
   'properties': {'name': 'Rhône - Porte du Scex (2009)',
    'link': 'http://www.hydrodaten.admin.ch/en/2009.html'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [7.3579, 46.2191], 'type': 'Point'},
   'id': '2011',
   'properties': {'name': 'Rhône - Sion (2011)',
    'link': 'http://www.hydrodaten.admin.ch/en/2011.html'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [8.1949, 47.4825], 'type': 'Point'},
   'id': '2016',
   'properties': {'name': 'Aare - Brugg (2016)',
    'link': 'http://www.hydrodaten.admin.ch/en/2016.html'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [8.2713, 47.421], 'type': 'Point'},
   'id': '2018',
   'properties': {'name': 'Reuss - Mellingen (2018)',
    'link': 'http://www.hydrodaten.admin.ch/en/2018.html'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [8.092, 46.7457], 'type': 'Point'},
   'id': '2019',
   'properties': {'name